In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)


In [ ]:
df = pd.read_csv("../input/aula-2-ia-dataset/CasasParaAlugar.csv").drop(columns='Unnamed: 0')

# Análise de dados

In [ ]:
df.head()

In [ ]:
# Analyzing variables types
variablesToAnalyze = ['bathroom', 'parking spaces', 'animal', 'furniture']
for vta in variablesToAnalyze:
    print(df[vta].unique())

In [ ]:
NominalCategorical = ['city', 'floor', 'animal', 'furniture']
OrdinalCategorical = []
Numeric = ['area', 'rooms', 'bathroom', 'parking spaces', 'hoa (R$)', 'rent amount (R$)', 'property tax (R$)', 'fire insurance (R$)', 'total (R$)']


In [ ]:
# missing data percentage
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0) * 100 / len(df)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(8, 10))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Porcentagem de variáveis ausentes por coluna")
plt.show()

In [ ]:
# Total percentage of missing data
totalMissingPercentage = df.isna()[df.isna() == True].count().sum() / (df.shape[0] * df.shape[1]) * 100
print(totalMissingPercentage, '%')

In [ ]:
# Skeweness
df.skew()

In [ ]:
# Proportion of qualitative data categories
NominalCategorical

In [ ]:
df['city']

In [ ]:
def groupOthers(dataFrame):
    total = dataFrame.value_counts().sum()
    bellow3Percent = dataFrame.value_counts()[dataFrame.value_counts() < total*0.03].index.values
    notOthers = dataFrame.value_counts()[dataFrame.value_counts() >= total*0.03].index.values
    result = dataFrame.value_counts()[notOthers]
    if len(bellow3Percent) > 0:
        result['Others'] = dataFrame.value_counts()[bellow3Percent].sum()
    return pd.DataFrame(result)
    
def plotPieChart(dataFrame):
    # Pie chart, where the slices will be ordered and plotted counter-clockwise:
    labels = dataFrame.index.values
    sizes = np.squeeze(dataFrame.values, axis=1)

    fig1, ax1 = plt.subplots()
    ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
            shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    plt.show()
    
for nc in NominalCategorical:
    plotPieChart(groupOthers(df[nc]))

In [ ]:
univariableAnalysis = pd.DataFrame(df.mean(), columns=['Mean'])
univariableAnalysis['Median'] = df.median()
#univariableAnalysis['Mode'] = df.mode()
univariableAnalysis['Var'] = df.var()
univariableAnalysis['Skew'] = df.skew()
univariableAnalysis['Max'] = df.max()
univariableAnalysis['Min'] = df.min()
univariableAnalysis

In [ ]:
df

In [ ]:
# Correcting floor and defining string features
df['floor'] = df['floor'].replace('-', '0').astype('float64')
stringVars = ['city', 'animal', 'furniture']

In [ ]:
df.describe()

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(15, 15))
# Quando criamos graficos com multiplas dimensoes, axes vira um array 2D. Então
# vamos transformar ele numa lista para iterar durante a criação do grafico
axes = axes.flatten()

# Iterando de grafico em grafico
for i,ax in enumerate(axes):
    if i < len(df.columns):
        col = df.columns[i]
        if col in stringVars:
            ax.bar(df[col].value_counts().index.values, df[col].value_counts().values)
        else:
            ax.hist(df[col])
        ax.set_title(col)

plt.tight_layout()

In [ ]:
import math
# comparando variaveis com total
varsToCompare = list(filter(lambda col: 'R$' not in col and col not in stringVars, df.columns))

fig, axes = plt.subplots(nrows=math.ceil(len(varsToCompare)/2), ncols=2, figsize=(15, 15))
# Quando criamos graficos com multiplas dimensoes, axes vira um array 2D. Então
# vamos transformar ele numa lista para iterar durante a criação do grafico
axes = axes.flatten()

# Iterando de grafico em grafico
for i,ax in enumerate(axes):
    if i < len(varsToCompare):
        col = varsToCompare[i]
        ax.scatter(x=df[col], y=df['total (R$)'])
        ax.set_title(col + ' X total (R$)')

plt.tight_layout()

# Limpeza de dados

In [ ]:
# Correcting floor - value
df['floor'].replace({'-':'0'})

In [ ]:
# missing data percentage
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0) * 100 / len(df)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(8, 10))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Porcentagem de variáveis ausentes por coluna")
plt.show()

In [ ]:
# usando regressao para valores numericos
df_imput_regress = pd.concat([df['total (R$)'], df['fire insurance (R$)'], df['property tax (R$)'] , df['rent amount (R$)'], df['hoa (R$)']], axis=1)
df_imput_regress.head()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Criamos um objeto que fará a Imputação por Regressão
imp_mean = IterativeImputer(random_state=0)
# Treinamos a regressão com os dados disponiveis
imp_mean.fit(df_imput_regress.values)

# Agora, faremos uma regressão nos mesmos dados usados no treinamento, para
# gerar valores numéricos para substituir os valores ausentes de LotFrontage
X = df_imput_regress.values
regr_output = imp_mean.transform(X)
regr_output

In [ ]:
df['total (R$)'] = regr_output[:, 0]
df['fire insurance (R$)'] = regr_output[:, 1]
df['property tax (R$)'] = regr_output[:, 2]
df['rent amount (R$)'] = regr_output[:, 3]
df['hoa (R$)'] = regr_output[:, 4]

In [ ]:
# missing data percentage
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0) * 100 / len(df)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(8, 10))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Porcentagem de variáveis ausentes por coluna")
plt.show()

In [ ]:
missingDataSeries = list(filter(lambda col: df[col].isna().sum(axis=0) > 0, df.columns))
missingDataSeries

In [ ]:
# replacing missing data with the value that appears the most if it appears >= 70% times in the series
for mds in missingDataSeries:
    counter = df[mds].value_counts()
    if counter.iloc[0] >= df[mds].notna().sum() * 0.7:
        df[mds] = df[mds].fillna(counter.index[0])
        print('Replaced missing ' + str(mds) + ' values with ' + str(counter.index[0]))

In [ ]:
# missing data percentage
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0) * 100 / len(df)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(8, 10))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Porcentagem de variáveis ausentes por coluna")
plt.show()

In [ ]:
# usando regressao para valores numericos
df_imput_regress = pd.concat([df['area'], df['rooms'], df['bathroom']], axis=1)
df_imput_regress.head()

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Criamos um objeto que fará a Imputação por Regressão
imp_mean = IterativeImputer(random_state=0)
# Treinamos a regressão com os dados disponiveis
imp_mean.fit(df_imput_regress.values)

# Agora, faremos uma regressão nos mesmos dados usados no treinamento, para
# gerar valores numéricos para substituir os valores ausentes de LotFrontage
X = df_imput_regress.values
regr_output = imp_mean.transform(X)
regr_output

In [ ]:
df['area'] = regr_output[:, 0]
df['rooms'] = regr_output[:, 1]
df['bathroom'] = regr_output[:, 2]

In [ ]:
# missing data percentage
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0) * 100 / len(df)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(8, 10))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Porcentagem de variáveis ausentes por coluna")
plt.show()

In [ ]:
# removing registers with null floor, parking spaces and cities
df = df.dropna(subset=['floor', 'parking spaces', 'city'])

In [ ]:
# missing data percentage
# Funcao do Pandas usada para contar o numero de valores vazios de cada coluna
data = df.isna().sum(axis=0) * 100 / len(df)
y = list(range(df.shape[1]))
x = data.values

# Criamos uma figura
fig, ax = plt.subplots(figsize=(8, 10))

# Plota as barras
ax.barh(y=y, width=x)

# Adiciona informações no gráfico
ax.set_yticks(y)
ax.set_yticklabels(df.columns.values)
ax.set_title("Porcentagem de variáveis ausentes por coluna")
plt.show()

# Outliers

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(15, 15))
# Quando criamos graficos com multiplas dimensoes, axes vira um array 2D. Então
# vamos transformar ele numa lista para iterar durante a criação do grafico
axes = axes.flatten()

# Iterando de grafico em grafico
for i,ax in enumerate(axes):
    if i < len(df.columns):
        col = df.columns[i]
        if col in stringVars:
            ax.bar(df[col].value_counts().index.values, df[col].value_counts().values)
        else:
            ax.hist(df[col])
        ax.set_title(col)

plt.tight_layout()

In [ ]:
varsToCompare = list(filter(lambda col: 'R$' in col, df.columns))
varsToCompare.remove('total (R$)')
varsToCompare.append('area')
fig, axes = plt.subplots(nrows=1, ncols=len(varsToCompare), figsize=(15, 5))

boxplotProps = []

for i,col in enumerate(varsToCompare):
    boxplotProps.append(axes[i].boxplot(df[col]))
    axes[i].set_title(col)

plt.tight_layout()

In [ ]:
def getMaxAndMinWhiskers(i):
    return [item.get_ydata()[1] for item in boxplotProps[i]['whiskers']]
len(boxplotProps)

In [ ]:
print("Tamanho do dataset antes dos filtros: {}".format(df.shape))

for i, col in enumerate(varsToCompare):
    
    minLimit, maxLimit = getMaxAndMinWhiskers(i)
    print (col, minLimit, maxLimit)
    
    mask = df[col] < maxLimit
    df = df[mask]
    mask = df[col] >= max(minLimit, 0)
    df = df[mask]

print("Tamanho do dataset depois dos filtros: {}".format(df.shape))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=len(varsToCompare), figsize=(15, 5))

boxplotProps = []

for i,col in enumerate(varsToCompare):
    boxplotProps.append(axes[i].boxplot(df[col]))
    axes[i].set_title(col)

plt.tight_layout()

# Feature Engineering

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=3, figsize=(15, 15))
# Quando criamos graficos com multiplas dimensoes, axes vira um array 2D. Então
# vamos transformar ele numa lista para iterar durante a criação do grafico
axes = axes.flatten()

# Iterando de grafico em grafico
for i,ax in enumerate(axes):
    if i < len(df.columns):
        col = df.columns[i]
        if col in stringVars:
            ax.bar(df[col].value_counts().index.values, df[col].value_counts().values)
        else:
            ax.hist(df[col])
        ax.set_title(col)

plt.tight_layout()

In [ ]:
# Creating new features
df['totalXarea'] = df.T.apply(lambda x: x['total (R$)'] / x['area'])
df['totalXrooms'] = df.T.apply(lambda x: x['total (R$)'] / x['rooms'])
df['totalXbathroom'] = df.T.apply(lambda x: x['total (R$)'] / x['bathroom'])
df['totalXfloor'] = df.T.apply(lambda x: x['total (R$)'] / (x['floor'] + 1))

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(15, 15))
# Quando criamos graficos com multiplas dimensoes, axes vira um array 2D. Então
# vamos transformar ele numa lista para iterar durante a criação do grafico
axes = axes.flatten()

# Iterando de grafico em grafico
for i,ax in enumerate(axes):
    if i < len(df.columns):
        col = df.columns[i]
        if col in stringVars:
            ax.bar(df[col].value_counts().index.values, df[col].value_counts().values)
        else:
            ax.hist(df[col])
        ax.set_title(col)

plt.tight_layout()

# Feature Select

In [ ]:
selected_columns = pd.Series(list(filter(lambda col: col not in stringVars, df.columns)))
df[selected_columns].shape

In [ ]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
pd.DataFrame(sel.fit_transform(df[selected_columns])).shape

# Rescaling

In [ ]:
df[selected_columns].describe()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = df[selected_columns]
scaler.fit(data)

In [ ]:
# Reescalonamos os dados
data_scaled = scaler.transform(data)

# Criamos um dataframe para facilitar a visualização
data_scaled = pd.DataFrame(data_scaled)
# "Devolvemos" os nomes das features e os índices para o dataframe
data_scaled.columns = selected_columns
data_scaled.index = df.index

data_scaled.describe()